# Motivation und Definition
Frequent Pattern Mining oder manchmal auch Frequent Pattern Discovery bezeichnet die Suche nach häufig gemeinsame auftretenden Mustern oder Items innerhalb großer Datenmengen.

In [ ]:
from tui_dsmt.fpm import BruteForceFI, receipts

## Inhaltsverzeichnis
- [Anwendung: Warenkorbanalyse](#Anwendung-Warenkorbanalyse)
- [Begriff: Assoziationsregel](#Begriff-Assoziationsregel)
- [Definitionen](#Definitionen)
- [Assoziationsregeln](#Assoziationsregeln)
- [Trivialer Algorithmus zum Auffinden von Assoziationsregeln](#Trivialer-Algorithmus-zum-Auffinden-von-Assoziationsregeln)

## Anwendung: Warenkorbanalyse
Zur Umgestaltung eines (fiktiven) Supermarktes soll die Effizienz erhöht und damit die Artikel so eingeräumt werden, dass häufig gekaufte Dinge nah beieinander platziert sind. Dazu werden alle von den Kassen generierten Bons zunächst in eine Transaktionsdatenbank geladen und anschließend untereinander verglichen, um häufige vorkommende Kombinationen zu finden. Diese Bons können beispielsweise wie folgt aussehen:

| Bon ID | Artikel                           |
| ------ | --------------------------------- |
| 1      | Brot, Butter, Milch, Zucker       |
| 2      | Butter, Mehl, Milch, Zucker       |
| 3      | Butter, Eier, Milch, Salz         |
| 4      | Eier                              |
| 5      | Butter, Mehl, Milch, Salz, Zucker |

## Begriff: Assoziationsregel
Eine Assoziationsregel ist eine Regel in der folgenden Form.
$$
\text{Rumpf} \rightarrow \text{Kopf} [\text{support}, \text{confidence}]
$$

Der *Support* bezeichnet dabei die Häufigkeit der Kombination *(Rumpf, Kopf)* in Relation zu allen Transaktionen. Mit Hilfe des Supports können also Regeln ausgeschlossen werden, die in wenigen Transaktionen vorkommen und somit auf Einzelereignisse zurückzuführen sind. Die *Konfidenz* dagegen bezeichnet die Häufigkeit des Auftretens der Kombination *(Rumpf, Kopf)* in Relation zu allen Transaktionen, die *Kopf* enthalten. Die Konfidenz dient damit dem Ausschluss von Regeln, die nur einen schwachen Zusammenhang beschreiben.

Die Regel
$$
\text{Windeln} \rightarrow \text{Bier} [0.5\%, 60\%]
$$
sagt demnach aus, dass $0.5\%$ aller Kassenbons die Kombination Bier und Windeln enthält und in $60\%$ aller Käufe von Windeln auch Bier gekauft wurde.

## Definitionen
- **Literale** bilden die Grundlage, indem sie einzelne Positionen bezeichnen. Sie sind die kleinste Einheit und bezeichnen beispielsweise einzelne Artikel.
- Die Menge **Items** $I = \{i_1, \dots, i_m\}$ bezeichnet die Menge der Literale.
- Ein **Itemset** $X \subseteq I$ ist eine spezifische Menge von Items, z.B. der Inhalt eines bestimmten Kassenbons.
- Eine **Datenbank** ist eine Menge von Transaktionen $\{T | T \subseteq I\}$.

Für die nachfolgenden Algorithmen ist es notwendig, dass die Items innerhalb der Itemsets eine lexikographische Sortierung besitzen.

Weiterhin wird die Länge des Itemsets als die Anzahl der Elemente innerhalb dieses Itemsets definiert. Ein Itemset der Länge $k$ wird außerdem als $k$-Itemset bezeichnet.

## Assoziationsregeln
- Eine **Assoziationsregel** ist eine Implikation der Form $X \Rightarrow Y$, wobei $X \subseteq I$, $Y \subseteq I$ und $X \cap Y = \emptyset$.
- Der **Support einer Menge** $X$ in $D$ ist die Anzahl der Transaktionen in $D$, die $X$ enthalten: $$
\sigma(X) = |\{ t_i | X \subseteq t_i, t_i \in T \}|
$$
- Der **Support einer Assoziationsregel** $X \Rightarrow Y$ ist der Anteil des gemeinsamen Auftretens von $X$ und $Y$ in Relation zur Anzahl aller Transaktionen: $$
s(X \Rightarrow Y) = \frac{\sigma(X \cup Y)}{|T|}
$$
- Die **Konfidenz einer Assoziationsregel** $X \Rightarrow Y$ ist der Anteil des gemeinsamen Auftretens von $X$ und $Y$ in Relation zur Anzahl des Auftretens von $X$: $$
c(X \Rightarrow Y) = \frac{\sigma(X \cup Y)}{\sigma(X)}
$$

## Trivialer Algorithmus zum Auffinden von Assoziationsregeln
Im Allgemeinen teilen sich die Algorithmen zum Auffinden entsprechender Assoziationsregeln in folgende Schritte:
1. Die häufig auftretenden Itemsets werden in der Datenbank gesucht. Häufig auftretend bedeutet dabei, dass der Support höher als ein vorgegebener, minimaler Support sein muss.
2. Aus den gefundenen häufigen Itemsets werden Assoziationsregeln generiert.

Im Folgenden wird ein trivialer Algorithmus beschrieben, der ohne besondere Annahmen auskommt.

### Suche nach häufigen Itemsets
Der einfachste Algorithmus arbeitet wie folgt:
1. Alle Teilmengen $D$ von $I$ werden generiert.
2. Für jede dieser Teilmengen $D$ wird die Häufigkeit in der Datenbank gezählt und die verworfen, die den minimalen Support nicht erfüllen.
3. Für jede verbleibende Teilmenge wird erneut jede Teilmenge $A \subseteq D$ betrachtet und geprüft, ob die Assoziationsregel $A \rightarrow (D \backslash A)$ den minimalen Support erfüllt.

Die nachfolgende Zelle enthält ein reduziertes Beispiel, das mit Hilfe des trivialen Algorithmus abgearbeitet wird. Bereits der eingangs gezeigte Transaktionsdatensatz würde die Darstellung zu stark vergrößeren.

In [ ]:
bffi = BruteForceFI(receipts, min_supp=2)
bffi

### Suche nach Assoziationsregeln
Jede verbleibende Teilmenge ist nun häufig. Der Algorithmus arbeitet weiter:

3. Für jedes häufige Itemset $X$ werden alle Teilmenge $A \subset X$ betrachtet. Geprüft wird dann, ob die Assoziationsregel $A \rightarrow (X \setminus A)$ den minimalen Support erfüllt. ($1$-Itemsets entfallen in dieser Betrachtung!)

In [ ]:
bffi.association_rules()

Offensichtlich lässt sich dieser Algorithmus aber nur für sehr kleine Mengen Datenbanken verwenden, da $2^{|I|}-1$ Teilmengen geprüft werden müssen. Die nachfolgenden Abschnitte beschäftigen sich deshalb mit effizienteren Methoden, um häufige Itemsets und Assoziationsregeln zu finden.